# LangChain: Coding AI Agents in Python

#### By Pedro Izquierdo Lehmann

Welcome to this hands-on introduction to **LangChain**! This notebook will guide you through building intelligent AI agents that can use tools, remember conversations, and make decisions autonomously.

**What is LangChain?**
LangChain is a framework for developing applications powered by language models. With it you can build explicit **chains**, which is an abstraction of an algorithm involving LLMs calls. Also, LangChain promotes implicit chains: instead of just asking an LLM questions, you can give it **tools** to use, and it will intelligently decide when and how to use them to answer your questions, instead of writing complex routing logic. 

LangChain works with the abstraction of the objects involved in the agentic system, such as

- **Chains**: Abstraction of an algorithm involving multiple steps; a reusable workflow.
- **Agents**: Abstraction of an LLM model equipped with tools, which can decide which tools/steps to run (an implicit chain).
- **Tools**: Wrapped Python functions so the agent can call them.
- **Memory/State**: Abstraction of context across conversation.

LangChain orders these in **layers** of abstraction, so you can start simple and add power only when you need it. Each layer builds on the previous one. This notebook follows that same progression: we start with tools, then add memory, context, and structured outputs.

**Content:**
- Creating your first AI agent
- Building custom tools for agents to use
- Adding memory so agents remember past conversations
- Using structured output for consistent responses
- Context-aware tools that access user information
- Best practices for production-ready agents

Let's get started!

---
## (0. Environment Setup)

Before starting, you need to set up a Python virtual environment and install all required dependencies. Follow these steps:

#### 1. Create a Virtual Environment

Open your terminal and navigate to the **directory containing this notebook**, then run:

```bash
python3 -m venv lang-chain
```

This creates a virtual environment in a folder called `lang-chain`.

#### 2. Activate the Virtual Environment

**On macOS/Linux:**
```bash
source lang-chain/bin/activate
```

**On Windows:**
```bash
lang-chain\Scripts\activate
```

You should see `(lang-chain)` at the beginning of your terminal prompt, indicating the virtual environment is active.

#### 3. Install Required Dependencies

With the virtual environment activated, install all necessary packages:

```bash
pip install langchain langgraph langchain-anthropic langchain-openai jupyter ipykernel
```

This will install:
- `langchain` - The core LangChain framework
- `langgraph` - For building stateful agent workflows and checkpointers
- `langchain-anthropic` - Anthropic (Claude) model provider
- `langchain-openai` - OpenAI model provider
- `jupyter` - Jupyter notebook environment
- `ipykernel` - Jupyter kernel for the virtual environment

Register the virtual environment as a Jupyter kernel:

```bash
python -m ipykernel install --user --name=lang-chain --display-name "Python (lang-chain)"
```

This ensures Jupyter can use your virtual environment's Python interpreter.

#### 4. Start Jupyter Notebook

We recommend two options to run the notebook:

**Jupyter Notebook:**

```bash
jupyter notebook
```

This will open Jupyter in your web browser. Navigate to and open this notebook (`LangChain.ipynb`).

**Code Editor like VS Code or Cursor:**

1. Open the notebook file (`LangChain.ipynb`) in your code editor
2. The editor should automatically detect it as a Jupyter notebook
3. When prompted to select a kernel, choose **Python (lang-chain)** from the list
4. If the kernel doesn't appear, you may need to refresh the kernel list or ensure the virtual environment is properly registered

#### 5. Deactivate

Don't forget to deactivate the virtual environment when you're done working with the following command:

```bash
deactivate
```

In [ ]:
# python3 -m venv lang-chain
# source lang-chain/bin/activate
# lang-chain\Scripts\activate
# pip install langchain langgraph langchain-anthropic langchain-openai jupyter ipykernel
# python -m ipykernel install --user --name=lang-chain --display-name "Python (lang-chain)"
# jupyter notebook

In [2]:
# Import necessary libraries
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from dataclasses import dataclass, field
import os

# Set API key
import getpass
import os

# Prompt user for the API key securely
api_key = getpass.getpass("Enter your OpenAI API Key: ")

# Set environment variable (recommended for LangChain)
os.environ["OPENAI_API_KEY"] = api_key


---

## 1. Chains

A **chain** is a sequence of steps (prompts, tools, or other chains) connected into a **single reusable pipeline**.

- Think of it as a recipe: each step transforms the input and passes it to the next.
- Chains can be simple (prompt -> LLM) or complex (multi-step reasoning + tools).
- Agents *use* chains internally, but chains are **deterministic**: the steps are predefined.

LangChain lets you build **chains explicitly** (deterministic pipelines) or **implicitly** through agents (dynamic pipelines).

- **Explicit chain:** You wire together steps (prompt → model → parsing). The flow is fixed and repeatable.
- **Agentic (implicit) chain:** The model decides which steps/tools to run at runtime. The flow can vary across calls. 

> **Note**: The cool thing about agent chains is that instead of just asking an LLM a question, you can give it **tools** to use, and it will decide when and how to use them to answer your question. For example, you don't need to write code that says "if the user asks about weather, call the weather tool." The agent figures this out on its own!

Below is an example of an explicit chain.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# Explicit chain: fixed four-step pipeline
# Step 1: Prompt template
chain_prompt = ChatPromptTemplate.from_messages([
    (
        "system", # sets the global rule for output format: title line + exactly three bullets. It’s treated as higher‑priority instructions.
        "Return output with first line 'Title: {topic}', followed by exactly three bullet points."
    ),
    (
        "human", # supplies the task input (the specific topic) and adds a content constraint (bullets must be full sentences).
        "Topic: {topic}. Each bullet must be a complete sentence."
    )
])

# Step 2: Model call
chain_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)

# Step 3: Parse to string
parser = StrOutputParser() # converts the raw LLM output into a string.

# Step 4: Deterministic formatting
format_output = RunnableLambda(
    lambda s: "\n".join(
        [line for line in [
            ("Title: options pricing" if not s.strip().split("\n")[0].startswith("Title:") else s.strip().split("\n")[0]),
            *[
                (line if line.strip().startswith("-") else f"- {line.strip()}")
                for line in s.strip().split("\n")[1:]
                if line.strip()
            ][:3]
        ] if line]
    )
)

chain = chain_prompt | chain_model | parser | format_output
result = chain.invoke({"topic": "options pricing"})
print(result)

Title: options pricing
- Options pricing involves determining the fair value of a financial derivative based on the underlying asset's price, volatility, time to expiration, and other factors.  
- The Black-Scholes model is a widely used mathematical framework for calculating the theoretical price of European-style options.  
- Market conditions and supply-demand dynamics can cause actual option prices to deviate from their theoretical values.


### Exercise 1: Build an Explicit Chain

Create a chain that produces **three bullet points** about a finance topic. Use an explicit prompt + model pipeline, then invoke it.

Hint: Use `ChatPromptTemplate`, compose with `|`, and access `response.content`.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# EXERCISE: Build an explicit chain
# 1. Create a ChatPromptTemplate with a {topic} variable
# 2. Initialize a model with temperature=0
# 3. Compose the chain with |
# 4. Invoke it with topic="risk-neutral pricing"
# 5. Print response.content

prompt = ChatPromptTemplate.from_template("Tell me three bullet points about {topic}.")  # TODO: Fill this in
model = init_chat_model("gpt-4o-mini", temperature=0, api_key=api_key)  # TODO: Fill this in
chain = prompt | model  # TODO: Fill this in
response = chain.invoke({"topic": "risk-neutral pricing"})  # TODO: Fill this in

print(response)

content='Sure! Here are three key points about risk-neutral pricing:\n\n1. **Concept of Risk Neutrality**: In risk-neutral pricing, investors are assumed to be indifferent to risk, meaning they require no additional return for taking on risk. This simplifies the valuation of financial derivatives by allowing the use of expected payoffs discounted at the risk-free rate.\n\n2. **Use in Derivative Pricing**: Risk-neutral pricing is a fundamental principle in the pricing of derivatives, such as options. The Black-Scholes model, for example, employs risk-neutral valuation to determine the fair price of options by calculating the expected payoff under a risk-neutral measure and discounting it back to present value.\n\n3. **Arbitrage-Free Pricing**: The risk-neutral pricing framework is built on the concept of no-arbitrage, which states that in an efficient market, there should be no opportunities to make a riskless profit. This principle ensures that the prices of financial instruments refle

In the following will introduce chains implicitly as we build tool-driven workflows, then show how agents extend them with decision-making.

## 2. Creating an Agent

Let's start by creating a simple agent. You can think of an agent as a **chain with decision-making**: it interprets the user input, decides which tools to call (if any), and produces a final response.

An agent needs:
1. A **model** (the LLM that does the thinking)
2. **Tools** (functions the agent can call)
3. A **system prompt** (instructions for the agent)

Here's a concrete example of how to create and use an agent with a time tool:

In [8]:
# Example: Create an agent with a time tool
def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in a specified timezone."""
    from datetime import datetime
    return f"Current time in {timezone}: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Initialize the model
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)

# Create the agent
example_agent = create_agent(
    model=example_model,
    tools=[get_current_time],
    system_prompt="You are a helpful time assistant"
)

# Invoke the agent
example_response = example_agent.invoke(
    {"messages": [{"role": "user", "content": "what time is it?"}]}
)

# Access the response
print("Example response:", example_response['messages'][-1].content)

Example response: The current time in UTC is 16:59 on January 16, 2026.


### Exercise 2: Create a Basic Agent

Now it's your turn! Create your first agent following the example above:

In [10]:
# First, let's create a simple tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# EXERCISE: Create a basic agent with a weather tool
# 1. Initialize a chat model
# Hint: Use init_chat_model() from langchain.chat_models with model name "gpt-4.1-nano-2025-04-14" or "gpt-4"
model = init_chat_model("gpt-4o-mini", temperature=0, api_key=api_key)  # TODO: Fill this in

# 2. Create an agent using create_agent
# Hint: Use create_agent() from langchain.agents with model, tools=[get_weather], and system_prompt="You are a helpful assistant"
agent = create_agent(model, tools=[get_weather], system_prompt="You are a helpful assistant")  # TODO: Fill this in

# 3. Run the agent with a message asking about the weather in San Francisco
# Hint: Use agent.invoke() with {"messages": [{"role": "user", "content": "what is the weather in San Francisco"}]}
response = agent.invoke({"messages": [{"role": "user", "content": "what is the weather in San Francisco"}]})  # TODO: Fill this in

# Print the response
print(response)

{'messages': [HumanMessage(content='what is the weather in San Francisco', additional_kwargs={}, response_metadata={}, id='0a6b607d-9f0e-4d55-85f3-3dad03b42168'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cym9IEJ3wzE7mNfUR7sYChtelzPXA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bc8d2-8d37-7573-8bb8-1f681a025b6f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_SZMgw436qLhi8rWnEuZkzfXY', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens'

## 3. Creating Custom Tools

Tools are functions that agents can call. LangChain makes it easy to convert Python functions into tools using the `@tool` decorator. The `@tool` decorator:
- Automatically extracts function name, description, and parameters
- Makes the function available to the agent
- Handles type validation and conversion

> **Important**: The function's docstring becomes part of the agent's prompt! Make it descriptive so the agent knows when to use the tool.

Here's a working example using different tools (string manipulation) to demonstrate the @tool decorator:

In [11]:
# Example: Create tools for string manipulation (different from the calculator exercise)
from langchain.tools import tool

@tool
def reverse_string(text: str) -> str:
    """Reverse a string."""
    return text[::-1]

@tool
def uppercase_string(text: str) -> str:
    """Convert a string to uppercase."""
    return text.upper()

@tool
def count_words(text: str) -> int:
    """Count the number of words in a string."""
    return len(text.split())

# Create an agent with these string manipulation tools
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)
string_agent = create_agent(
    model=example_model,
    tools=[reverse_string, uppercase_string, count_words],
    system_prompt="You are a helpful text processing assistant"
)

# Test the agent
example_response = string_agent.invoke(
    {"messages": [{"role": "user", "content": "Reverse the string 'Hello World' and count its words"}]}
)
print("Example response:", example_response['messages'][-1].content)

Example response: The reversed string is "dlroW olleH" and it contains 2 words.


### Exercise 3: Create a LangChain tools

Now it's your turn! Create your first LangChain tools following the syntaxis of the example above:

In [12]:
### Exercise 2: Create Multiple Tools

# EXERCISE: Create a calculator agent with multiple tools
# 1. Create a tool for addition
# Hint: Use @tool decorator from langchain.tools, function should take (a: float, b: float) and return a + b
@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

# 2. Create a tool for multiplication
# Hint: Use @tool decorator, function should take (a: float, b: float) and return a * b
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# 3. Create a tool for getting the square root
# Hint: Use @tool decorator and import math, function should take (x: float) and return math.sqrt(x)
import math
@tool
def sqrt(x: float) -> float:
    """Calculate the square root of a number."""
    return math.sqrt(x)

# 4. Create an agent with all three tools
# Hint: Use create_agent() from langchain.agents with model, tools=[add, multiply, sqrt], and system_prompt="You are a helpful calculator assistant"
calculator_agent = create_agent(
    model=model, 
    tools=[add, multiply, sqrt], 
    system_prompt="You are a helpful calculator assistant"
)

# 5. Test your agent
# Hint: Use calculator_agent.invoke() with {"messages": [{"role": "user", "content": "What is 15 plus 27, then multiply that by 3?"}]}
response = calculator_agent.invoke({"messages": [{"role": "user", "content": "What is 15 plus 27, then multiply that by 3?"}]})
print(response)

{'messages': [HumanMessage(content='What is 15 plus 27, then multiply that by 3?', additional_kwargs={}, response_metadata={}, id='b18dd942-69ca-4054-a89a-4635e901b307'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 108, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3683ee3deb', 'id': 'chatcmpl-CymAbayjxE0tEVOXGevoThZlxRSfV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bc8d3-cc93-7f43-83ec-88f3d157212d-0', tool_calls=[{'name': 'add', 'args': {'a': 15, 'b': 27}, 'id': 'call_77crbX2OJY0t6hoS29pR4mol', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 108

**Expected Behavior**: The agent should:
1. First call `add(15, 27)` to get 42
2. Then call `multiply(42, 3)` to get 126
3. Return the final answer

This demonstrates that agents can **chain multiple tool calls** to solve complex problems! The agent automatically figures out the sequence of operations needed.

## 4. Tools with Runtime Context

Sometimes tools need access to runtime information (like user IDs, session data, etc.). LangChain provides `ToolRuntime` for this. `ToolRuntime` allows tools to access:
- **Context**: Custom data passed when invoking the agent
- **Memory**: Conversation history and state
- **Configuration**: Runtime settings

> **Note**: The `ToolRuntime` parameter is automatically injected by LangChain. You don't pass it when calling the tool - LangChain handles that for you!

Here is an example that uses `ToolRuntime` to build Context-Aware Tools

In [13]:
# # Example: Context-aware tool for user preferences (different from greeting exercise)
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

# Define a context schema with user preferences
@dataclass
class UserContext:
    """Custom runtime context schema."""
    user_id: str
    favorite_color: str

# Create a tool that uses ToolRuntime to access user preferences
@tool
def get_recommendation(runtime: ToolRuntime[UserContext]) -> str:
    """Get a personalized recommendation based on user preferences."""
    user_id = runtime.context.user_id
    favorite_color = runtime.context.favorite_color
    return f"User {user_id} might like items in {favorite_color} color!"

# Create an agent with this tool
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)
recommendation_agent = create_agent(
    model=example_model,
    tools=[get_recommendation],
    system_prompt="You are a helpful recommendation assistant",
    context_schema=UserContext
)

# Invoke with context
example_response = recommendation_agent.invoke(
    {"messages": [{"role": "user", "content": "What would you recommend for me?"}]},
    context=UserContext(user_id="123", favorite_color="blue")
)
print("Example response:", example_response['messages'][-1].content)

Example response: Based on your preferences, I recommend exploring items in blue color. Would you like some specific suggestions or categories to consider?


### Exercise 3: Context-Aware Tools

Now create your own personalized greeting tool:

In [14]:
# EXERCISE: Create a personalized greeting tool using runtime context
# 1. Define a Context dataclass with a user_name field
# Hint: Use @dataclass from dataclasses, create a class Context with user_name: str field
@dataclass
class Context:
    user_name: str

# 2. Create a tool that uses ToolRuntime to access context
# Hint: Use @tool from langchain.tools, function parameter should be runtime: ToolRuntime[Context], access user_name via runtime.context.user_name
@tool
def get_personalized_greeting(runtime: ToolRuntime[Context]) -> str:
    """Get a personalized greeting based on the user's name."""
    return f"Hello, {runtime.context.user_name}!"

# 3. Create an agent with this tool
# Hint: Use create_agent() with model, tools=[get_personalized_greeting], system_prompt, and context_schema=Context
personalized_agent = create_agent(
    model=model,
    tools=[get_personalized_greeting],
    system_prompt="You are a helpful assistant who greets people by name.",
    context_schema=Context
)

# 4. Invoke the agent with context
# Hint: Use personalized_agent.invoke() with messages and context=Context(user_name="Alice")
response = personalized_agent.invoke(
    {"messages": [{"role": "user", "content": "Please greet me."}]},
    context=Context(user_name="Alice")
)
print(response)

{'messages': [HumanMessage(content='Please greet me.', additional_kwargs={}, response_metadata={}, id='dd1d4773-8920-4b44-8ae4-a34f5c02f2d5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 61, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CymDGv0AlogKEEcNhUnNX9ab1G4Mz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bc8d6-4da3-7563-8d16-394a9f5dff05-0', tool_calls=[{'name': 'get_personalized_greeting', 'args': {}, 'id': 'call_1kPQ2yfkeHlKxEWCcvNxXF9o', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 61, 'output_tokens': 14, 't

## 5. Adding Memory to Agents

So far, our agents don't remember previous conversations. Let's add **memory** so agents can maintain context across multiple interactions. A **checkpointer** stores conversation state, which you can think of as the **state of the agent's chain across turns**. LangChain provides:
- `InMemorySaver`: For development/testing (lost when program ends)
- Database checkpointers: For production (persistent storage)

> **Note**: `ToolRuntime` and `InMemorySaver` both relate to “runtime context,” but they operate at different layers.
>- `ToolRuntime` is per-tool-call and injected into tool functions; it provides a view of context/memory/config at that moment.
>- `InMemorySaver` is storage, passed to the agent as a checkpointer to persist conversation state between invocations (keyed by `thread_id`). It does not get injected into tools.
>- `ToolRuntime` doesn’t store anything by itself; `InMemorySaver` doesn’t provide arbitrary runtime context/config—only persistence for state.

Here is an example of how to add memory to an agent:

In [15]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool

# Create a checkpointer
example_checkpointer = InMemorySaver()

# Create a quote tool (different from the fact tool in the exercise)
@tool
def get_quote(category: str) -> str:
    """Get an inspirational quote by category."""
    quotes = {
        "success": "Success is not final, failure is not fatal: it is the courage to continue that counts.",
        "wisdom": "The only true wisdom is in knowing you know nothing.",
        "motivation": "The way to get started is to quit talking and begin doing."
    }
    return quotes.get(category.lower(), "Here's a quote: Keep moving forward!")

# Create an agent with memory
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)
quote_agent = create_agent(
    model=example_model,
    tools=[get_quote],
    system_prompt="You are a helpful assistant that shares inspirational quotes",
    checkpointer=example_checkpointer
)

# Create a config with thread_id
example_config = {"configurable": {"thread_id": "quote-session-1"}}

# First message
example_response1 = quote_agent.invoke(
    {"messages": [{"role": "user", "content": "Give me a quote about success"}]},
    config=example_config
)

# Second message - agent remembers!
example_response2 = quote_agent.invoke(
    {"messages": [{"role": "user", "content": "What quote did you just share?"}]},
    config=example_config
)
print("First response:", example_response1['messages'][-1].content)
print("Second response:", example_response2['messages'][-1].content)

First response: Here's an inspiring quote about success: "Success is not final, failure is not fatal: it is the courage to continue that counts."
Second response: I shared the quote: "Success is not final, failure is not fatal: it is the courage to continue that counts."


> **Note**: The `thread_id` in the config is crucial! It tells the checkpointer which conversation to load. Different `thread_id` values mean different conversations. This allows you to manage multiple concurrent conversations with the same agent.

### Exercise 4: Conversational Memory

Now create your own agent with memory:

In [17]:
# EXERCISE: Create an agent with conversational memory
# 1. Create an InMemorySaver checkpointer
# Hint: Import InMemorySaver from langgraph.checkpoint.memory and create an instance
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()  # TODO: Fill this in

# 2. Create a simple tool that returns a fact
@tool
def get_fact(topic: str) -> str:
    """Get an interesting fact about a topic."""
    facts = {
        "python": "Python was named after Monty Python's Flying Circus",
        "ai": "The term 'artificial intelligence' was coined in 1956",
        "space": "A day on Venus is longer than its year"
    }
    return facts.get(topic.lower(), f"I don't know much about {topic}")

# 3. Create an agent with the checkpointer
# Hint: Use create_agent() with model, tools=[get_fact], system_prompt, and checkpointer=checkpointer
memory_agent = create_agent(
    model=model,
    tools=[get_fact],
    system_prompt="You are a helpful assistant.",
    checkpointer=checkpointer
)  # TODO: Fill this in

# 4. Create a config with a thread_id (this identifies the conversation)
# Hint: Create a dictionary with {"configurable": {"thread_id": "conversation-1"}}
config = {"configurable": {"thread_id": "conversation-1"}}  # TODO: Fill this in

# 5. Ask the agent: "Tell me a fact about Python"
# Hint: Use memory_agent.invoke() with messages and config
response1 = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me a fact about Python"}]},
    config=config
)  # TODO: Fill this in
print("First response:", response1['messages'][-1].content)

# 6. In a follow-up message, ask: "What was the fact you just told me?"
# Hint: Use memory_agent.invoke() again with the same config - the agent should remember!
response2 = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What was the fact you just told me?"}]},
    config=config
)  # TODO: Fill this in
print("Second response:", response2['messages'][-1].content)

First response: A fun fact about Python is that it was named after "Monty Python's Flying Circus," the British comedy television show.
Second response: The fact I shared is that Python was named after "Monty Python's Flying Circus," the British comedy television show.


## 6. Structured Output

Sometimes you want the agent's response in a specific format. LangChain supports **structured output** using dataclasses or `Pydantic` models. Its functional object for this is `ToolStrategy`, which tells the agent to use tools AND return structured output. The agent will still use tools, but format its final response according to your schema. This gives you the best of both worlds - tool usage with predictable output formats.

Here's how to create an agent with structured output:

In [18]:
from dataclasses import dataclass, field
from langchain.agents.structured_output import ToolStrategy
from langchain.tools import tool

# Define a different response format
@dataclass
class ProductRecommendation:
    """Response schema for product recommendations."""
    product_name: str
    price: float
    rating: float = 0.0
    features: list[str] = field(default_factory=list)

# Create a product search tool
@tool
def search_products(category: str) -> str:
    """Search for products in a category."""
    return f"Found products in {category}: Laptop ($999, 4.5 stars), Tablet ($499, 4.2 stars)"

# Create an agent with structured output
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)
product_agent = create_agent(
    model=example_model,
    tools=[search_products],
    system_prompt="You are a helpful product recommendation assistant",
    response_format=ToolStrategy(ProductRecommendation)
)

# Ask a question and get a structured response
example_response = product_agent.invoke(
    {"messages": [{"role": "user", "content": "Recommend a laptop for me"}]}
)

# Access the structured response
structured = example_response['structured_response']
print("Product Name:", structured.product_name)
print("Price:", structured.price)
print("Rating:", structured.rating)
print("Features:", structured.features)

Product Name: Laptop
Price: 999.0
Rating: 4.5
Features: ['High performance', 'Lightweight design', 'Long battery life']


### Exercise 5: Structured Responses

Now create your own agent with structured output:

In [19]:
# EXERCISE: Create an agent with structured output
# 1. Define a ResponseFormat dataclass
# Hint: Use @dataclass from dataclasses, include answer: str, confidence: float = 0.0, and sources: list[str] = field(default_factory=list)
from dataclasses import dataclass, field

@dataclass
class ResponseFormat:
    answer: str
    confidence: float = 0.0
    sources: list[str] = field(default_factory=list)

# 2. Create a simple tool
# Hint: Use @tool from langchain.tools, function should take (query: str) and return a string
@tool
def search_knowledge_base(query: str) -> str:
    """Search the internal knowledge base for information."""
    return "LangChain is a framework for developing applications powered by language models."

# 3. Create an agent with structured output
# Hint: Use create_agent() with model, tools, system_prompt, and response_format=ToolStrategy(ResponseFormat) from langchain.agents.structured_output
from langchain.agents.structured_output import ToolStrategy

structured_agent = create_agent(
    model=model,
    tools=[search_knowledge_base],
    system_prompt="You are a helpful research assistant.",
    response_format=ToolStrategy(ResponseFormat)
)

# 4. Ask a question and get a structured response
# Hint: Use structured_agent.invoke() with messages
response = structured_agent.invoke({"messages": [{"role": "user", "content": "What is LangChain?"}]})

# Access the structured response
structured = response['structured_response']
print(f"Answer: {structured.answer}")
print(f"Confidence: {structured.confidence}")
print(f"Sources: {structured.sources}")

Answer: LangChain is a framework for developing applications powered by language models. It provides tools and components to facilitate the integration of language models into various applications, enabling developers to create more sophisticated and interactive experiences using natural language processing.
Confidence: 0.9
Sources: []


## 7. Guided Exercise: Daily S&P 500 Decision with Twitter Sentiment Analysis

In this exercise, you will build an agent that decides whether to **BUY** or **NOT BUY** units of the S&P 500 (e.g., SPY), computing **local sentiment** for tweets **before the decision day**. We use the following Hugging Face dataset: https://huggingface.co/datasets/StephanAkkerman/stock-market-tweets-data

> **Note**: This is a simplified educational example; don’t take it as financial advice.



In [ ]:
from langchain.tools import tool
from datasets import load_dataset
import pandas as pd
import re
import os

# Build a local pandas dataframe (sampled)
ds = load_dataset("StephanAkkerman/stock-market-tweets-data", split="train")
df = ds.select(range(20000)).to_pandas()

# Normalize and parse dates
df.columns = [c.strip() for c in df.columns]
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
df = df.dropna(subset=["created_at", "text"])
df["created_at_date"] = df["created_at"].dt.date

In [ ]:
# Local sentiment scoring
positive_words = {"gain", "gains", "bull", "bullish", "up", "upgrade", "beat", "strong", "rally", "surge", "record"}
negative_words = {"loss", "losses", "bear", "bearish", "down", "downgrade", "miss", "weak", "selloff", "drop", "plunge"}

def sentiment_score(text: str) -> int:
    tokens = re.findall(r"[a-zA-Z']+", text.lower())
    score = sum(1 for t in tokens if t in positive_words) - sum(1 for t in tokens if t in negative_words)
    return score

df["sentiment_score"] = df["text"].fillna("").apply(sentiment_score)

# Define the sentiment summary tool
# Hint: Use @tool from langchain.tools. Compute statistics from df["sentiment_score"] (e.g. average).
@tool
def get_sentiment_summary() -> str:
    """Summarize local tweet sentiment across the full dataset."""
    ... # TODO: Fill this in

# Write the SYSTEM_PROMPT
# Hint: Include goals + rules; Summary must include average sentiment score and key themes.
SYSTEM_PROMPT = None # TODO: Fill this in

# Initialize the model
example_model = init_chat_model("gpt-4.1-nano-2025-04-14", temperature=0, api_key=api_key)

# Create the agent and run it
# Hint: create_agent(model=example_model, tools=[get_sentiment_summary], system_prompt=SYSTEM_PROMPT)
decision_agent = None  # TODO: Fill this in

response = decision_agent.invoke(
    {"messages": [{"role": "user", "content": "Use the overall tweet sentiment to decide BUY or NOT BUY SPY."}]})
print(response["messages"][-1].content)



## Congratulations 
You've completed the LangChain tutorial! We covered

- How to create agents with LangChain  
- How to build custom tools  
- How to add memory to agents  
- How to use structured output  
- How to build a daily decision agent  

### Possible next steps to explore
   - **LangGraph**: For more complex agent workflows (see the LangGraph notebook!)
   - **Retrieval**: Connect agents to vector databases for RAG
   - **Multi-agent systems**: Agents that collaborate
   - **LangSmith**: Observability and debugging tools

### Additional resources
   - [LangChain Docs](https://docs.langchain.com)
   - [LangChain Quickstart](https://docs.langchain.com/oss/python/langchain/quickstart)





Happy learning!

Pedro